In [86]:
import tweepy
import sys
import os
import pandas as pd
from optparse import OptionParser
import os.path
import re
import numpy as np

## Regex Pattern for email matching
regex = re.compile(("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                    "{|}~-]+)*(@)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                    "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))

## API KEYs and SECRETs from twitter developer account
API_KEY = "B42ivEiaVpxFkkQ8Xkhve5OPI"
API_SECRET = "DK2LQlooUt4MYTNTE10ehw7YhOJjLd8TvjNfs0o2ElrySGFxMr"
ACCESS_KEY = "87912348-Ot0hIDjawWv9cA1gdqcnOU7hSYhNMYlgHOuTJcerY"
ACCESS_SECRET = "Kj014Z9n1w61pnHJ4jkikky1tvB2nIQjHtkIdRqNMBuLZ"

def get_emails(s):
    """Returns an iterator of matched emails found in string s"""
    for e in (email[0] for email in re.findall(regex, s) if not email[0].startswith('//')):
        if e != None:
            return e
    return "NA"

In [10]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [72]:
search_query = '#mentalhealth OR #mindfulness'  # this is what we're searching for
max_tweets = 100000
tweets_per_qry = 100
fName = 'tweets.txt'
max_id = -1
since_id = None
data = pd.DataFrame(columns = ["screen_name", "user_name", "follower_count",
                               "friends_count", "email", "is_verified", "lang"])

In [73]:
def update_data(data, k):
    #update = pd.read_csv("data.csv", encoding ="utf8")
    #update = pd.concat([update,data])
    #update = update.drop_duplicates("screen_name")
    data.to_csv("data" + str(k) + ".csv", index=False, encoding="utf8")

def get_tweets(data, since_id, max_id, search_query, tweets_per_qry):
    total_max_id = 0
    tweet_count = 0
    k = 1
    while tweet_count < max_tweets:
        data_inter = pd.DataFrame(columns = ["screen_name", "user_name",
                                             "follower_count", "friends_count",
                                             "email", "is_verified", "lang"])
        try: 
            if (max_id <= 0):
                if (not since_id):
                    new_tweets = api.search(q=search_query, count=tweets_per_qry)
                    total_max_id = new_tweets[0].id
                else:
                    new_tweets = api.search(q=search_query, count=tweets_per_qry,
                                            since_id=since_id)
            else:
                if (not since_id):
                    new_tweets = api.search(q=search_query, count=tweets_per_qry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=search_query, count=tweets_per_qry,
                                            max_id=str(max_id - 1),
                                            since_id=since_id)
            i=0 

            for tweet in new_tweets:
                temp = tweet._json
                email = get_emails(str(temp['user']["description"]).lower())
                # print (email)
                data_inter.loc[i] = [temp['user']["screen_name"],
                                     temp['user']["name"],temp['user']["followers_count"],
                                     temp['user']["friends_count"], email,
                                     temp['user']["verified"], temp['user']["lang"]]
                i = i + 1

            tweet_count += len(new_tweets) 

            print("{0} tweets".format(tweet_count))
            max_id = new_tweets[-1].id
            data = pd.concat([data, data_inter])

            ## keep writing down intermediate dataframe. after each iteation of 10000
            if tweet_count >= k*5000:    
                if (k==1):
                    data.to_csv("data" + str(k) + ".csv", index=False, encoding="utf8")
                    data = data[0:0]
                    data_inter = data_inter[0:0]
                    k=k+1
                else:
                    update_data(data, k)
                    k=k+1
                    data = data[0:0]
                    data_inter = data_inter[0:0]

        except tweepy.TweepError as e:
            print("error : " + str(e))
            break
    
    update_data(data, k)
    return total_max_id

In [74]:
total_max_id = get_tweets(data, since_id, max_id, search_query, tweets_per_qry)

100 tweets
200 tweets
297 tweets
392 tweets
482 tweets
581 tweets
681 tweets
781 tweets
881 tweets
981 tweets
1081 tweets
1181 tweets
1281 tweets
1381 tweets
1481 tweets
1581 tweets
1681 tweets
1781 tweets
1881 tweets
1981 tweets
2081 tweets
2181 tweets
2281 tweets
2381 tweets
2481 tweets
2581 tweets
2681 tweets
2781 tweets
2881 tweets
2981 tweets
3081 tweets
3181 tweets
3281 tweets
3381 tweets
3481 tweets
3581 tweets
3681 tweets
3781 tweets
3881 tweets
3981 tweets
4081 tweets
4181 tweets
4281 tweets
4381 tweets
4481 tweets
4581 tweets
4681 tweets
4781 tweets
4881 tweets
4981 tweets
5081 tweets
5181 tweets
5281 tweets
5381 tweets
5481 tweets
5581 tweets
5681 tweets
5781 tweets
5881 tweets
5981 tweets
6081 tweets
6181 tweets
6281 tweets
6381 tweets
6481 tweets
6581 tweets
6681 tweets
6781 tweets
6881 tweets
6981 tweets
7081 tweets
7181 tweets
7281 tweets
7381 tweets
7481 tweets
7581 tweets
7681 tweets
7781 tweets
7881 tweets
7981 tweets
8081 tweets
8181 tweets
8281 tweets
8381 tweets
84

In [76]:
files = !ls data*.csv
data = pd.DataFrame()
for f in files:
    temp = pd.read_csv(f, encoding="utf8")
    data = pd.concat([data,temp])

In [78]:
data = data.drop_duplicates("screen_name")

In [81]:
data.head()

,screen_name,user_name,follower_count,friends_count,email,is_verified,lang
0,MaryS19994064,Mary S,67.0,115.0,NaN,False,en
1,DrTuckey,Michelle Tuckey,253.0,220.0,NaN,False,en
2,breaking_ts,Breaking the Silence,2545.0,2861.0,NaN,False,en
3,mentalbravery,Simple Reminders,16.0,0.0,NaN,False,en
4,kibokogroup,Kiboko Group of Co.,317.0,461.0,NaN,False,en


In [94]:
## Filling missing values in email
data["email"] = data["email"].fillna("NA")

In [97]:
## Data with email IDs
data[data["email"] != "NA"]

,screen_name,user_name,follower_count,friends_count,email,is_verified,lang
18,lonely_illness,The Lonely Illness,174.0,293.0,thelonelyillness@gmail.com,False,en
32,sudeepkhatri,sudeep,4896.0,2615.0,sudeep.khatri8090@gmail.com,False,en
81,MarinaIlieva1,Simply Marina,3645.0,3662.0,marina.ilieva7@gmail.com,False,en
268,iam_young_alpha,Master P🌍,2536.0,4824.0,aphiwexabanisa@gmail.com,False,en
301,TELWellness,The Essential Life,18409.0,138.0,info@theessentiallife.org,False,en
483,LindaLeeKing,Linda Lee King,35483.0,20779.0,lindaleeking1950@gmail.com,False,en
702,Treasure_E_M,Helen,2878.0,1777.0,treasure-every-moment@hotmail.co.uk,False,en
1392,KateGillieART,Kate Gillie,12662.0,11069.0,kate@kategillieart.com,False,en
1685,KSL5TV,KSL 5 TV,61364.0,262.0,social@ksl.com,True,en
1835,MaxDonatelli,Max E Donatelli,243.0,422.0,maxjoydonatelli@gmail.com,False,en


In [103]:
## data with follower counts between 1000-50000
data_influencer = data[(data["follower_count"] >= 1000) & (data["follower_count"] <= 50000)]

In [104]:
data.to_csv("data.csv", index=False)

In [105]:
data_influencer.to_csv("data_influencer.csv", index=False)

In [107]:
data_contacts = data[data["email"] != "NA"]
data_contacts.to_csv("data_contact.csv", index=False)